# NanoSafety data exploration



In [ ]:
import yaml
from solrscope import aa
from solrscope import units
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import requests
from importlib import reload 
from solrscope import client_solr
from solrscope import client_ambit
from solrscope import annotation
import pandas as pd
import numpy as np
import json
import pandas as pd
import importlib
import qgrid
import warnings
import seaborn as sns

import matplotlib.pyplot as plt
import io
import qgrid
from measurement.utils import guess
from measurement.measures import Distance, Temperature, Volume, Area, Frequency, Mass, Time
#init_notebook_mode(connected=True)
warnings.simplefilter("ignore")

In [ ]:
print('Select enanoMapper aggregated search service:')
style = {'description_width': 'initial'}
config,config_servers, config_security, auth_object, msg = aa.parseOpenAPI3()    
service_widget = widgets.Dropdown(
    options=config_servers['url'],
    description='Service:',
    disabled=False,
    style=style
)
if config_security is None:
    service = interactive(aa.search_service_open,url=service_widget)
else:
    print(msg)
    apikey_widget=widgets.Text(
            placeholder='',
            description=config_security,
            disabled=False,
            style=style
    )    
    service = interactive(aa.search_service_protected,url=service_widget,apikey=apikey_widget)    

display(service)

In [ ]:
service_uri=service_widget.value
if auth_object!=None:
    auth_object.setKey(apikey_widget.value)
print("Sending queries to {}".format(service_uri))
facets = client_solr.Facets()


##  Study query

In [ ]:
    query_widget=widgets.Text(
        placeholder='',
        value="NANOREG",
        description="Query",
        disabled=False,
        style=style
    )
    display(query_widget)

In [ ]:
reload(client_solr)
facets = client_solr.Facets()

f_method="E.method_s"
f_materialtype="substanceType_s"
f_materialtypename="substanceType_name"
f_materialname="publicname_s"
f_value="Number of data points"



df = facets.summary(service_uri,auth_object, query=query_widget.value,fields=["topcategory_s","endpointcategory_s","owner_name_s",f_method,f_materialtype,f_materialname])    
A = df.groupby([f_materialtypename,f_materialname]).agg({f_method: "count", f_value : sum}).reset_index()
#A
B = A.groupby([f_materialtypename]).agg({f_materialname : "count", f_method: sum, f_value : sum}).reset_index()
qgrid.show_grid(df)
#qgrid.show_grid(A)
#qgrid.show_grid(B)

In [ ]:
from matplotlib import rcParams
rcParams['figure.figsize'] = 11.7,8.27
#sns.set(rc={'figure.figsize':(10,10)})
dict = annotation.DictionarySubstancetypes()
ax = sns.scatterplot(x=f_method, y=f_value, size=f_materialname,hue=f_materialtypename,data=B, sizes=(200, 2000)
#                     ,hue_order=dict.lookup.properties.values()
                    )
ax.legend(loc='center left', bbox_to_anchor=(1.25, 0.5), ncol=1)
plt.title(query_widget.value)
# Set x-axis label
plt.xlabel("Method")

for line in range(0,B.shape[0]):
     if B[f_materialname][line]>12:
         ax.text(B[f_method][line]+0.01, B[f_value][line], 
         B.substanceType_name[line], horizontalalignment='left', 
         size='medium', color='black')

## Get the study results

In [ ]:

url=service_uri + "/select?"
maxrows=100000
q = {"fl": "name:name_s,material_name:publicname_s,project:owner_name_s,material_type_annotation:substanceType_s,category:topcategory_s,endpointcategory_s,protocol:guidance_s,descriptor:effectendpoint_s,descriptor_type:effectendpoint_type_s,descriptor_annotation:effectendpoint_synonym_ss,provider:reference_owner_s,citation:reference_s,citation_year:reference_year_s,text_value:textValue_s,loValue:loValue_d,upValue:upValue_d,uncertainty:err_value_d,unit:unit_s,method:E.method_s,method_annotation:E.method_synonym_ss,descriptor_annotation:effectendpoint_synonym_ss,textValue:textValue_s",
      "fq":"type_s:study","wt":"csv", "rows":maxrows,
     "q":query_widget.value

#     "q":"(endpointcategory_s:PC_GRANULOMETRY_SECTION) AND (owner_name_s:SANOWORK)"
#       "q":"endpointcategory_s:SPECIFIC_SURFACE_AREA_SECTION"
#       "q":"endpointcategory_s:DUSTINESS_SECTION"     
#       "q":"endpointcategory_s:CRYSTALLINE_PHASE_SECTION"          
#       "q":"endpointcategory_s:POROSITY_SECTION"          
#        "q":"endpointcategory_s:AGGLOMERATION_AGGREGATION_SECTION"          
#        "q":"endpointcategory_s:PC_DENSITY_SECTION"               
#        "q":"endpointcategory_s:TO_GENETIC_IN_VITRO_SECTION"             
#        "q":"TO_GENETIC_IN_VITRO_SECTION  AND COMET"                
#         "q" : "ENM_0000068_SECTION"
#         "q" : "NPO_1339_SECTION"
#         "q" : "TO_REPEATED_ORAL_SECTION"
#         "q" : "TO_REPEATED_INHAL_SECTION"
#         "q" : "ENM_0000037_SECTION"     
    }
r = client_solr.get(url,query=q,auth=auth_object)
df=pd.read_csv(io.StringIO(r.content.decode('utf-8')))
df.head()

try:
    a = annotation.DictionarySubstancetypes()
    df[ 'material_type']=df[ 'material_type_annotation'].apply(a.annotate)
except:
    pass
    
a = annotation.DictionaryEndpointCategoryNames()
df[ 'endpoint']=df[ 'endpointcategory_s'].apply(a.annotate)



In [ ]:

qgrid_widget = qgrid.show_grid(df[['project','material_type','material_name','endpoint','method','method_annotation','descriptor','descriptor_annotation','descriptor_type','unit','loValue','upValue','uncertainty','textValue']])
qgrid_widget

### Unit conversion (if needed)

In [ ]:
df_filtered = qgrid_widget.get_changed_df()
uq = df_filtered["unit"].unique()
    
def convert_units_df(to_unit=None,debug=True):
    if to_unit==None:
        df_filtered['loValue_std'] =  df_filtered['loValue']
        df_filtered['upValue_std'] =   df_filtered['upValue']
        df_filtered['uncertainty_std'] = df_filtered['uncertainty']
        df_filtered['unit_std'] = df_filtered['unit']   
        if debug:
            display(df_filtered.head(3))
    else:    
        df_filtered['loValue_std'] = df_filtered.apply(lambda x: units.convert_units(x.loValue, x.unit,  to_unit), axis=1)
        df_filtered['upValue_std'] = df_filtered.apply(lambda x: units.convert_units(x.upValue, x.unit,to_unit), axis=1)
        df_filtered['uncertainty_std'] = df_filtered.apply(lambda x: units.convert_units(x.uncertainty, x.unit, to_unit), axis=1)
        df_filtered['unit_std'] = to_unit
        
        
        if debug:
            display(df_filtered[df_filtered["unit"]!=to_unit].head(3))
    
    return to_unit


convert_units_df(None,debug=False)
try:
    interact(convert_units_df, to_unit=uq);
except Exception as err:    
    print(err)
    pass;

# Boxplot

In [ ]:
x_widget = widgets.Dropdown(
    options= df_filtered.select_dtypes(include=[object]),
    description='X:',
    value='material_type',
    disabled=False
)
y_widget = widgets.Dropdown(
    options= ["loValue_std","upValue_std","uncertainty_std","loValue","upValue","uncertainty"],
    description='Y:',
    value='loValue_std',
    disabled=False
)
hue_widget = widgets.Dropdown(
    options= df_filtered.select_dtypes(include=[object]),
    description='hue:',
    value='descriptor',
    disabled=False
)
col_widget = widgets.Dropdown(
    options= df_filtered.select_dtypes(include=[object]),
    description='col:',
    value='endpoint',
    disabled=False
)


In [ ]:
sns.set(style="ticks")
sns.set(rc={'figure.figsize':(20,10)})
def striplot(x,y, hue, boxplot):
    try:    
        if boxplot:
            ax = sns.boxplot(x,y, data=df_filtered, whis=np.inf)
        ax = sns.stripplot(x,y,  hue=hue, dodge=True, data=df_filtered, jitter=True)
        ax.legend(loc='center left', bbox_to_anchor=(1.25, 0.5), ncol=1)
        ylabel = '{} [{}]'.format(','.join(df_filtered['descriptor'].unique()), ','.join(df_filtered['unit_std'].unique()))
        print(ylabel)
        ax.ylabel(ylabel)
        ax.xlabel(x)
    except:
        pass
interact(striplot,x= x_widget, y=y_widget, hue = hue_widget, boxplot=True);    

## Heat map

In [ ]:
p = df_filtered.pivot_table(index=["endpoint","method","descriptor","unit_std"],columns=["material_type","material_name"],values=[y_widget.value])
p

In [ ]:
sns.set(rc={'figure.figsize':(10,10)})
sns.heatmap(p)


## Endpoints

In [ ]:
import matplotlib.pyplot as plt
binary = ["#ff0000","#00ff00"]
sns.set_palette(binary)
#red/green palette

  
def plot_heatmap(table,q, cmap=binary,save=False):    
    sns.palplot(cmap)
    sns.set_style('ticks')
    fig, ax = plt.subplots()
    fig.set_size_inches(10,10)
    sns_plot = sns.heatmap(table, cmap=cmap)
    sns_plot.set_title(q)
    if save:
        sns_plot.get_figure().savefig(q+".png")
    return (sns_plot)

In [ ]:
def queryEndpoints(top="TOX",q="*:*"):
    try:
        df = facets.summary(service_uri,auth_object,query="(topcategory_s:"+top + ") AND (" + q + ")",fq="type_s:study",statistics="Number of data points", fields=["topcategory_s","endpointcategory_s","effectendpoint_s","substanceType_s","publicname_s"])        
        df.rename(columns={"owner_name_s" : "Project","topcategory_s":"Study type","substanceType_name": "NM type","endpointcategory_name":"endpoint"},inplace=True)
        table = pd.pivot_table(df, values="Number of data points", index=['NM type'], columns=['endpoint' ], aggfunc='any').fillna(False)
        plot_heatmap(table,q)
    except:
        pass
    #display(df.head())
    #display(table)
    
    

interact(queryEndpoints,top=["TOX","P-CHEM","ECOTOX","ENV FATE"],q="*:*")

## Composition

In [ ]:
url=service_uri + "/select?"

maxrows=10000
q = {"fl" : "s_uuid_hs,component_s,ChemicalName_s,CASRN_s,SMILES_s",
      "fq":"type_s:composition","wt":"csv", "rows":maxrows,
     "q":"*:*"
     
    }
r = client_solr.get(url,query=q,auth=auth_object)
df=pd.read_csv(io.StringIO(r.content.decode('utf-8')))
qgrid.show_grid(df)